In [4]:
import torch
from torchvision import models, transforms
from PIL import Image
import json
import base64
from io import BytesIO


def model_fn(model_dir):
    # 모델 파일 경로 설정
    model_path = f"{model_dir}/best_model_34.pth"
    print(f"Loading model from: {model_path}")

    try:
        # GPU가 있는지 확인
        if torch.cuda.is_available():
            device = torch.device("cuda")
            print(f"Model will run on GPU: {torch.cuda.get_device_name(0)}")
        else:
            device = torch.device("cpu")
            print("No GPU available. Model will run on CPU.")

        model = models.resnet34(pretrained=False)
        model.fc = torch.nn.Linear(model.fc.in_features, 2)  # 클래스 개수: 2개

        # 학습된 가중치 로드 (디바이스에 맞게 로드)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)  # 모델을 디바이스로 이동
        model.eval()  # 평가 모드로 설정
        print("Model loaded successfully.")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        raise e

In [5]:
model = model_fn('./')  # `best_model_34.pth` 파일이 있는 로컬 디렉토리에서 실행

Loading model from: .//best_model_34.pth
Model will run on GPU: Tesla T4


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model loaded successfully.


In [6]:
import time

start_time = time.time()
model = model_fn('./')
print(f"Model loading time: {time.time() - start_time} seconds")

Loading model from: .//best_model_34.pth
Model will run on GPU: Tesla T4
Model loaded successfully.
Model loading time: 0.43827223777770996 seconds


In [10]:
def input_fn(request_body, request_content_type):
    """
    SageMaker가 호출하는 입력 데이터 처리 함수
    """
    if request_content_type == "application/json":
        # JSON 데이터에서 이미지 경로 추출
        input_data = json.loads(request_body)
        #인코딩된 이미지데이터 디코딩
        image_data = base64.b64decode(input_data["image"])
        #디코딩된 데이터 PIllow이미지로 변환
        image = Image.open(BytesIO(image_data)).convert("RGB")
        return image
    else:
        raise ValueError(f"Unsupported content type: {request_content_type}")

In [11]:
# 모델 로드
model = model_fn('./')

# 테스트 입력 데이터 (Base64 인코딩된 JSON 파일을 생성한 경우)
with open('sample.json', 'r') as f:
    input_data = f.read()

# 입력 처리
image = input_fn(input_data, "application/json")

Loading model from: .//best_model_34.pth
Model will run on GPU: Tesla T4
Model loaded successfully.


In [17]:
def predict_fn(input_data, model):
    """
    SageMaker가 호출하는 추론 함수
    """
    # 데이터 전처리
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),  # ResNet 입력 크기로 조정
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # 전처리
    image_tensor = preprocess(input_data).unsqueeze(0)  # 배치 차원 추가

    # 모델과 동일한 디바이스로 이동
    device = next(model.parameters()).device
    image_tensor = image_tensor.to(device)

    # 모델 추론
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)  # 가장 높은 확률의 클래스 반환
    return predicted.item()


In [18]:
# 추론
result = predict_fn(image, model)

In [19]:

def output_fn(prediction, response_content_type):
    """
    SageMaker가 호출하는 출력 데이터 포맷팅 함수
    """
    if response_content_type == "application/json":
        
        result = "OK" if prediction == 1 else "NG"
        
        return json.dumps({"result": result})  # JSON 형식으로 결과 반환
    else:
        raise ValueError(f"Unsupported response content type: {response_content_type}")
